In [1]:
import pygsheets
from oauth2client.service_account import ServiceAccountCredentials
from pathlib import Path
import pandas as pd
pd.set_option('mode.chained_assignment', None)

In [2]:
import pandas as pd
pd.options.display.max_columns = 0

In [3]:

# Load the credentials from the JSON file (replace with the path to your file)
sf_path = r'inductive-gift-355101-48518c54d576.json'
gc = pygsheets.authorize(service_account_file=sf_path)


In [4]:
def get_number_gsheet(service_file_path, spreadsheet_id, sheet_name):
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    wks = sh.worksheet_by_title(sheet_name)
    df = wks.get_as_df(
        numerize=False, value_render='UNFORMATTED_VALUE', empty_value=None)
    df = df.replace(to_replace='', value=None)

    return df

# %%
# lay du lieu tu googhesheet ma khong co du lieu so


def get_df_from_gsheet(service_file_path, spreadsheet_id, sheet_name, start=None):
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    wks = sh.worksheet_by_title(sheet_name)
    df = wks.get_as_df(start, numerize=False, empty_value=None)
    df = df.replace(to_replace='', value=None)
    df.drop_duplicates(inplace=True)

    return df

In [5]:
def get_data(sheet_name, start):
    spreadsheet_id = '1Bmm9GGP3QE5k_T8eODN3Hrh8ICzMgAazYkw0gWNRZDs'

    gc = pygsheets.authorize(service_file=sf_path)
    sh = gc.open_by_key(spreadsheet_id)
    wks = sh.worksheet_by_title(sheet_name)
    df = wks.get_as_df(start, numerize=False, value_render='UNFORMATTED_VALUE', empty_value=None)
    df = df.replace(to_replace='', value=None)
    # df.drop_duplicates(inplace=True)

    df2 = df.transpose()
    df2.iloc[:,1] = df2.iloc[:,1].ffill()
    df3 = df2.transpose()
    # Combine the two lists
    combined_list = [f"{a}|{b}" for a, b in zip(df3.iloc[1].to_list(), df3.iloc[2].to_list())]
    df3.columns = combined_list
    return df3

In [6]:
def extract_daily(df3, sheet_name):
    df_daily = df3.iloc[10:,:]
    df_daily2 = df_daily.melt(id_vars='None|None')
    df_daily2[['user', 'criteria']] =df_daily2['variable'].str.split('|', expand=True)
    df_daily2.rename(columns={'None|None':'report_date'}, inplace=True)
    df_daily2.drop(columns='variable', inplace=True)
    df_daily2.dropna(subset='value', inplace=True)
    df_daily3 = df_daily2.query("value!=0").pivot(columns='criteria', index=['user', 'report_date'], values='value').reset_index()
    df_daily3['report_date'] = df_daily3['report_date'].astype('str') + '/' + sheet_name
    df_daily3['report_date'] = pd.to_datetime(df_daily3['report_date'], format='%d/%m/%Y')
    # df_daily3.to_parquet(rf".\data\daily\{str(sheet_name).replace('/', '-')}.parquet", index=False )
    return df_daily3

In [41]:
def extract_target(df3, sheet_name):
    df_target = df3.iloc[3:10]
    df_target.rename(columns={"None|None":"item"}, inplace=True)
    df_target.query("item.isin(['Target', 'Daily'])", inplace=True)

    df_target2 = df_target.melt(id_vars='item')
    df_target2[['user', 'criteria']] =df_target2['variable'].str.split('|', expand=True)
    # df_target2.rename(columns={'item':'report_date'}, inplace=True)
    df_target2.drop(columns='variable', inplace=True)
    df_target2.dropna(subset='value', inplace=True)

    df_target2.drop_duplicates(subset=['item', 'user'], inplace=True)
    df_target3 = df_target2.query("value!=0").pivot(columns='item', index=['user'], values='value').reset_index()

    df_target3['report_month'] = '01/' + sheet_name
    df_target3['report_month'] = pd.to_datetime(df_target3['report_month'], format='%d/%m/%Y')
    # df_target3.to_parquet(rf".\data\target\{str(sheet_name).replace('/', '-')}.parquet", index=False )
    return df_target3

In [42]:
sheets = [
        '9/2024',
        # '8/2024',
        # '7/2024',
        # '6/2024',
        # '5/2024',
        # '4/2024',
        # '3/2024',
        # '2/2024',
        # '1/2024',
        # '12/2023',
        # '11/2023',
        # '10/2023',
        # '9/2023',
        # '8/2023',
        # '7/2023',
        # '6/2023',
        # '5/2023',
        # '4/2023',
        # '3/2023',
        # '2/2023',
        # '1/2023',
]
for sheet_name in sheets:
    df3 = get_data(sheet_name=sheet_name, start='A1')
    df_daily = extract_daily(df3, sheet_name)
    df_target = extract_target(df3, sheet_name)
    # Get start of month and number of days in the month
    start_of_month = df_target.loc[0,'report_month']
    num_days = start_of_month.days_in_month

    # Create a date range for each day of the month
    date_range = pd.date_range(start_of_month, periods=num_days, freq='D')
    df_target['report_date'] = [date_range]*len(df_target)
    df_target = df_target.explode('report_date')
    df_target['report_day'] = df_target['report_date'].dt.day
    df_target['no_of_day'] = num_days
    df_target['Daily'] = df_target['Daily'].astype(int)

    df_target['Target'] = df_target['Target'].astype(int)
    df_target['daily_target_norm'] =  df_target['Target']/df_target["no_of_day"]
    df_target['mtd_target_norm'] = df_target['daily_target_norm'] * df_target['report_day']
    dta = df_target.merge(df_daily, how='left', on=['user','report_date'])
    dta['Total'] = dta['Total'].fillna(0)
    dta['Total'] = pd.to_numeric(dta['Total'], errors='coerce')
    dta['mtd_actual'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Total'].cumsum()
    # danh dau ngay dai daily target
    dta['flg_daily']  = 0
    flt = dta['Total'] >= dta['Daily']
    dta.loc[flt, 'flg_daily'] = 1

    # danh dau ngay co tap luyen
    dta['flg_workout'] = 0
    flt = dta['Total'] >0
    dta.loc[flt, 'flg_workout'] = 1


    for col in ['Burpee', 'Core','Pushup', 'Run', 'Squat','flg_daily', 'flg_workout']:
        try:
            dta[col] = pd.to_numeric(dta[col], errors='coerce')
            dta[col] = dta[col].fillna(0)
            new_col = 'mtd_'+col
            dta[new_col] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])[col].cumsum()
        except Exception as e:
            print(e)
    # tinh toan accumulate
    # dta.to_parquet(rf".\data\daily\dta_{str(sheet_name).replace('/', '-')}.parquet", index=False )
    print(sheet_name)

TypeError: dtype 'Ninh|Total    object
Ninh|Total    object
dtype: object' not understood

In [43]:
sheet_name='9/2024'
df3 = get_data(sheet_name=sheet_name, start='A1')


In [44]:
df3.head()

,None|None,Sơn|Total,Sơn|Burpee,Sơn|Run,Sơn|Pushup,Sơn|Core,Sơn|Squat,Ninh|Total,Ninh|Burpee,Ninh|Run,Ninh|Pushup,Ninh|Core,Ninh|Squat,Ninh|Total,Ninh|Burpee,Ninh|Run,Ninh|Pushup,Ninh|Core,Ninh|Squat,Kiên Chu|Total,Kiên Chu|Burpee,Kiên Chu|Run,Kiên Chu|Pushup,Kiên Chu|Core,Kiên Chu|Squat,Trung|Total,Trung|Burpee,Trung|Run,Trung|Plank,Trung|Core,Trung|Squat,Đức|Total,Đức|Burpee,Đức|Run,Đức|Pushup,Đức|Core,Đức|Squat,Thảo|Total,Thảo|Burpee,Thảo|Run,...,Tuấn|Run,Tuấn|Pushup,Tuấn|Core,Tuấn|Squat,Hoàng|Total,Hoàng|Burpee,Hoàng|Run,Hoàng|Pushup,Hoàng|Core,Hoàng|Squat,Thiện|Total,Thiện|Burpee,Thiện|Run,Thiện|Pushup,Thiện|Core,Thiện|Squat,Kiên Trần|Total,Kiên Trần|Burpee,Kiên Trần|Run,Kiên Trần|Pushup,Kiên Trần|Core,Kiên Trần|Squat,Quỳnh|Total,Quỳnh|Burpee,Quỳnh|Run,Quỳnh|Pushup,Quỳnh|Core,Quỳnh|Squat,Hiệp|Total,Hiệp|Burpee,Hiệp|Run,Hiệp|Pushup,Hiệp|Core,Hiệp|Squat,Hiệp|Total,Hiệp|Burpee,Hiệp|Run,Hiệp|Pushup,Hiệp|Core,Hiệp|Squat
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,Sơn,Sơn,Sơn,Sơn,Sơn,Sơn,Ninh,Ninh,Ninh,Ninh,Ninh,Ninh,Ninh,Ninh,Ninh,Ninh,Ninh,Ninh,Kiên Chu,Kiên Chu,Kiên Chu,Kiên Chu,Kiên Chu,Kiên Chu,Trung,Trung,Trung,Trung,Trung,Trung,Đức,Đức,Đức,Đức,Đức,Đức,Thảo,Thảo,Thảo,...,Tuấn,Tuấn,Tuấn,Tuấn,Hoàng,Hoàng,Hoàng,Hoàng,Hoàng,Hoàng,Thiện,Thiện,Thiện,Thiện,Thiện,Thiện,Kiên Trần,Kiên Trần,Kiên Trần,Kiên Trần,Kiên Trần,Kiên Trần,Quỳnh,Quỳnh,Quỳnh,Quỳnh,Quỳnh,Quỳnh,Hiệp,Hiệp,Hiệp,Hiệp,Hiệp,Hiệp,Hiệp,Hiệp,Hiệp,Hiệp,Hiệp,Hiệp
2,None,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,Plank,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,...,Run,Pushup,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat,Total,Burpee,Run,Pushup,Core,Squat
3,Target,4000,None,None,None,None,None,4000,None,None,None,None,None,2300,None,None,None,None,None,800,None,None,None,None,None,800,None,None,None,None,None,800,None,None,None,None,None,1800,None,None,...,None,None,None,None,800,None,None,None,None,None,800,None,None,None,None,None,2600,None,None,None,None,None,1000,None,None,None,None,None,1400,None,None,None,None,None,800,None,None,None,None,None
4,Unstop max,None,None,None,None,None,None,None,80,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [46]:
df_target = df3.iloc[3:10]
df_target.rename(columns={"None|None":"item"}, inplace=True)


In [47]:
df_target

,item,Sơn|Total,Sơn|Burpee,Sơn|Run,Sơn|Pushup,Sơn|Core,Sơn|Squat,Ninh|Total,Ninh|Burpee,Ninh|Run,Ninh|Pushup,Ninh|Core,Ninh|Squat,Ninh|Total,Ninh|Burpee,Ninh|Run,Ninh|Pushup,Ninh|Core,Ninh|Squat,Kiên Chu|Total,Kiên Chu|Burpee,Kiên Chu|Run,Kiên Chu|Pushup,Kiên Chu|Core,Kiên Chu|Squat,Trung|Total,Trung|Burpee,Trung|Run,Trung|Plank,Trung|Core,Trung|Squat,Đức|Total,Đức|Burpee,Đức|Run,Đức|Pushup,Đức|Core,Đức|Squat,Thảo|Total,Thảo|Burpee,Thảo|Run,...,Tuấn|Run,Tuấn|Pushup,Tuấn|Core,Tuấn|Squat,Hoàng|Total,Hoàng|Burpee,Hoàng|Run,Hoàng|Pushup,Hoàng|Core,Hoàng|Squat,Thiện|Total,Thiện|Burpee,Thiện|Run,Thiện|Pushup,Thiện|Core,Thiện|Squat,Kiên Trần|Total,Kiên Trần|Burpee,Kiên Trần|Run,Kiên Trần|Pushup,Kiên Trần|Core,Kiên Trần|Squat,Quỳnh|Total,Quỳnh|Burpee,Quỳnh|Run,Quỳnh|Pushup,Quỳnh|Core,Quỳnh|Squat,Hiệp|Total,Hiệp|Burpee,Hiệp|Run,Hiệp|Pushup,Hiệp|Core,Hiệp|Squat,Hiệp|Total,Hiệp|Burpee,Hiệp|Run,Hiệp|Pushup,Hiệp|Core,Hiệp|Squat
3,Target,4000,None,None,None,None,None,4000,None,None,None,None,None,2300,None,None,None,None,None,800,None,None,None,None,None,800,None,None,None,None,None,800,None,None,None,None,None,1800,None,None,...,None,None,None,None,800,None,None,None,None,None,800,None,None,None,None,None,2600,None,None,None,None,None,1000,None,None,None,None,None,1400,None,None,None,None,None,800,None,None,None,None,None
4,Unstop max,None,None,None,None,None,None,None,80,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,Tổng,1140,840,0,40,300,390,1700,820,0,0,1760,0,0,0,0,0,0,0,0,0,0,0,98,0,269,178,0,13,0,0,135,25,0,100,70,75,790,330,0,...,0,240,0,0,0,0,0,0,0,0,190,50,7,0,0,0,1280,1280,0,0,0,0,351.5,0,0,563,40,150,360,0,3,0,600,0,0,0,0,0,0,0
6,%,0.285,None,None,None,None,None,0.425,None,None,None,None,None,0,None,None,None,None,None,0,None,None,None,None,None,0.33625,None,None,None,None,None,0.16875,None,None,None,None,None,0.438889,None,None,...,None,None,None,None,0,None,None,None,None,None,0.2375,None,None,None,None,None,0.492308,None,None,None,None,None,0.3515,None,None,None,None,None,0.257143,None,None,None,None,None,0,None,None,None,None,None
7,Thiếu,2860,None,None,None,None,None,2300,None,None,None,None,None,2300,None,None,None,None,None,800,None,None,None,None,None,531,None,None,None,None,None,665,None,None,None,None,None,1010,None,None,...,None,None,None,None,800,None,None,None,None,None,610,None,None,None,None,None,1320,None,None,None,None,None,648.5,None,None,None,None,None,1040,None,None,None,None,None,800,None,None,None,None,None
8,Daily,160,None,None,None,None,None,160,None,None,None,None,None,92,None,None,None,None,None,32,None,None,None,None,None,32,None,None,None,None,None,32,None,None,None,None,None,72,None,None,...,None,None,None,None,32,None,None,None,None,None,32,None,None,None,None,None,104,None,None,None,None,None,40,None,None,None,None,None,56,None,None,None,None,None,32,None,None,None,None,None
9,Count,6,None,None,None,None,None,7,None,None,None,None,None,0,None,None,None,None,None,1,None,None,None,None,None,6,None,None,None,None,None,3,None,None,None,None,None,7,None,None,...,None,None,None,None,0,None,None,None,None,None,1,None,None,None,None,None,8,None,None,None,None,None,5,None,None,None,None,None,6,None,None,None,None,None,0,None,None,None,None,None


In [50]:
df_target.columns

Index(['item', 'Sơn|Total', 'Sơn|Burpee', 'Sơn|Run', 'Sơn|Pushup', 'Sơn|Core',
       'Sơn|Squat', 'Ninh|Total', 'Ninh|Burpee', 'Ninh|Run', 'Ninh|Pushup',
       'Ninh|Core', 'Ninh|Squat', 'Ninh|Total', 'Ninh|Burpee', 'Ninh|Run',
       'Ninh|Pushup', 'Ninh|Core', 'Ninh|Squat', 'Kiên Chu|Total',
       'Kiên Chu|Burpee', 'Kiên Chu|Run', 'Kiên Chu|Pushup', 'Kiên Chu|Core',
       'Kiên Chu|Squat', 'Trung|Total', 'Trung|Burpee', 'Trung|Run',
       'Trung|Plank', 'Trung|Core', 'Trung|Squat', 'Đức|Total', 'Đức|Burpee',
       'Đức|Run', 'Đức|Pushup', 'Đức|Core', 'Đức|Squat', 'Thảo|Total',
       'Thảo|Burpee', 'Thảo|Run', 'Thảo|Pushup', 'Thảo|Core', 'Thảo|Squat',
       'An|Total', 'An|Burpee', 'An|Run', 'An|Pushup', 'An|Core', 'An|Squat',
       'Tuấn|Total', 'Tuấn|Burpee', 'Tuấn|Run', 'Tuấn|Pushup', 'Tuấn|Core',
       'Tuấn|Squat', 'Hoàng|Total', 'Hoàng|Burpee', 'Hoàng|Run',
       'Hoàng|Pushup', 'Hoàng|Core', 'Hoàng|Squat', 'Thiện|Total',
       'Thiện|Burpee', 'Thiện|Run', 'Thiện|Pu

In [49]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 3 to 9
Data columns (total 91 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item              7 non-null      object
 1   Sơn|Total         6 non-null      object
 2   Sơn|Burpee        1 non-null      object
 3   Sơn|Run           1 non-null      object
 4   Sơn|Pushup        1 non-null      object
 5   Sơn|Core          1 non-null      object
 6   Sơn|Squat         1 non-null      object
 7   Ninh|Total        6 non-null      object
 8   Ninh|Burpee       2 non-null      object
 9   Ninh|Run          1 non-null      object
 10  Ninh|Pushup       1 non-null      object
 11  Ninh|Core         1 non-null      object
 12  Ninh|Squat        1 non-null      object
 13  Ninh|Total        6 non-null      object
 14  Ninh|Burpee       1 non-null      object
 15  Ninh|Run          1 non-null      object
 16  Ninh|Pushup       1 non-null      object
 17  Ninh|Core         1 

In [52]:
flt = df_target['item'].isin(['Target', 'Daily'])
df_target = df_target[flt]

In [53]:

df_target2 = df_target.melt(id_vars='item')
df_target2[['user', 'criteria']] =df_target2['variable'].str.split('|', expand=True)
# df_target2.rename(columns={'item':'report_date'}, inplace=True)
df_target2.drop(columns='variable', inplace=True)
df_target2.dropna(subset='value', inplace=True)

df_target2.drop_duplicates(subset=['item', 'user'], inplace=True)
df_target3 = df_target2.query("value!=0").pivot(columns='item', index=['user'], values='value').reset_index()

df_target3['report_month'] = '01/' + sheet_name
df_target3['report_month'] = pd.to_datetime(df_target3['report_month'], format='%d/%m/%Y')

In [56]:
df_target3.columns

Index(['user', 'Daily', 'Target', 'report_month'], dtype='object', name='item')

In [30]:
df_daily = df3.iloc[10:,:]
df_daily2 = df_daily.melt(id_vars='None|None')
df_daily2[['user', 'criteria']] =df_daily2['variable'].str.split('|', expand=True)
df_daily2.rename(columns={'None|None':'report_date'}, inplace=True)
df_daily2.drop(columns='variable', inplace=True)
df_daily2.dropna(subset='value', inplace=True)
df_daily2

,report_date,value,user,criteria
0,1,120,Sơn,Total
1,2,0,Sơn,Total
2,3,80,Sơn,Total
3,4,0,Sơn,Total
4,5,80,Sơn,Total
...,...,...,...,...
1559,24,0,Ninh,Total
1560,25,0,Ninh,Total
1561,26,0,Ninh,Total
1562,27,0,Ninh,Total


In [25]:
df_daily2['report_date'] = df_daily2['report_date'].astype(str)
df_daily3 = df_daily2.query("value!=0").pivot(columns='criteria', index=['user', 'report_date'], values='value').reset_index()
df_daily3


criteria,user,report_date,Burpee,Core,Pushup,Run,Squat,Total
0,Hiếu,1,51,26,NaN,NaN,NaN,64
1,Hiếu,10,97,20,NaN,NaN,NaN,107
2,Hiếu,17,28,17,NaN,NaN,NaN,36.5
3,Hiếu,2,31,20,NaN,NaN,NaN,41
4,Hiếu,23,31,36,NaN,NaN,NaN,49
...,...,...,...,...,...,...,...,...
103,Đức,27,75,30,NaN,NaN,75,115
104,Đức,28,55,60,10,NaN,90,120
105,Đức,29,70,40,NaN,NaN,30,100
106,Đức,4,55,NaN,NaN,NaN,NaN,55


In [28]:
df_daily3[df_daily3['report_date'].isna()]

criteria,user,report_date,Burpee,Core,Pushup,Run,Squat,Total


In [31]:
df_daily2[df_daily2['report_date'].isna()]

,report_date,value,user,criteria
447,None,16,Kiên,Burpee


In [29]:
df_daily3[df_daily3['report_date']=='None']

criteria,user,report_date,Burpee,Core,Pushup,Run,Squat,Total
45,Kiên,None,16,NaN,NaN,NaN,NaN,NaN


In [15]:
file = 'data/dta_daily.parquet'
df = pd.read_parquet(file)

In [16]:
df['mtd_non_burpee'] =  df['mtd_Run']*20 + df['mtd_Pushup']/2 + df['mtd_Core']/2 + df['mtd_Squat']/3 + df['mtd_Plank']*14/2
df['mtd_actual'] = df['mtd_Burpee'] + df['mtd_non_burpee']

In [18]:
flt = (df["user"]=="Quỳnh") & (df['report_month']>='2023-06-01') \
    & ((df['mtd_Run']*20+df['mtd_Core']/2+df['mtd_Squat']/3)>(df['mtd_Burpee']+df['mtd_Pushup']/2))
df[flt]


,user,Daily,Target,report_month,report_date,report_day,no_of_day,daily_target_norm,mtd_target_norm,Burpee,Core,Plank,Pushup,Run,Squat,Total,mtd_actual,flg_daily,flg_workout,mtd_Burpee,mtd_Core,mtd_Pushup,mtd_Run,mtd_Squat,mtd_Plank,mtd_flg_daily,mtd_flg_workout,mtd_non_burpee


In [23]:
flt = (df["user"]=="Quỳnh") & (df['report_date']=='2024-09-30') \
#     & ((df['mtd_Run']*20+df['mtd_Core']/2+df['mtd_Squat']/3)>(df['mtd_Burpee']+df['mtd_Pushup']/2))
df[flt]


,user,Daily,Target,report_month,report_date,report_day,no_of_day,daily_target_norm,mtd_target_norm,Burpee,Core,Plank,Pushup,Run,Squat,Total,mtd_actual,flg_daily,flg_workout,mtd_Burpee,mtd_Core,mtd_Pushup,mtd_Run,mtd_Squat,mtd_Plank,mtd_flg_daily,mtd_flg_workout,mtd_non_burpee
3121,Quỳnh,40,1000,2024-09-01,2024-09-30,30,30,33.333333,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,293.5,0,0,0.0,40.0,447.0,0.0,150.0,0.0,4,5,293.5


In [19]:
df.loc[flt, 'mtd_actual'] = (df.loc[flt, 'mtd_Burpee'] + df.loc[flt, 'mtd_Pushup']/2)*2.5

In [22]:
file_folder = 'data/daily'
df = pd.DataFrame()
for file in Path(file_folder).glob('*.parquet'):
    dfx = pd.read_parquet(file)
    df = pd.concat(objs=[dfx, df], axis=0, ignore_index=True)

df = df.fillna(0)
for col in ['Burpee', 'Core','Pushup', 'Run', 'Squat','Plank']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(0)
    new_col = 'mtd_'+col
    df[new_col] = pd.to_numeric(df[new_col], errors='coerce')
    df[new_col] = df[new_col].fillna(0)
        
# print(df.info())

# tinh toan accumulate
df['mtd_non_burpee'] =  df['mtd_Run']*20 + df['mtd_Pushup']/2 + df['mtd_Core']/2 + df['mtd_Squat']/3 + df['mtd_Plank']*14/2
df['mtd_actual'] = df['mtd_Burpee'] + df['mtd_non_burpee']


In [24]:

# them dieu kien rang buoc ve so Burpee toi thieu
flt = df['mtd_non_burpee'] > df['mtd_Burpee']*1.5
df.loc[flt, 'mtd_actual'] = df.loc[flt, 'mtd_Burpee']*2.5


In [27]:
flt = (df["user"]=="Quỳnh") & (df['report_date']=='2024-09-30') \
#     & ((df['mtd_Run']*20+df['mtd_Core']/2+df['mtd_Squat']/3)>(df['mtd_Burpee']+df['mtd_Pushup']/2))
df[flt]


,user,Daily,Target,report_month,report_date,report_day,no_of_day,daily_target_norm,mtd_target_norm,Burpee,Core,Plank,Pushup,Run,Squat,Total,mtd_actual,flg_daily,flg_workout,mtd_Burpee,mtd_Core,mtd_Pushup,mtd_Run,mtd_Squat,mtd_Plank,mtd_flg_daily,mtd_flg_workout,mtd_non_burpee
3121,Quỳnh,40,1000,2024-09-01,2024-09-30,30,30,33.333333,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,293.5,0,0,0.0,40.0,447.0,0.0,150.0,0.0,4,5,293.5


In [ ]:


# chinh rule cho Hiep tu thang 01/2024
flt = (df['user']=='Hiệp') & (df['report_month'] >= '2024-01-01')
df.loc[flt, 'mtd_actual'] = df.loc[flt, 'mtd_Burpee'] + df.loc[flt, 'mtd_non_burpee']


# chinh rule cho Hiep tu thang 11/2023 - 12/2023
flt = (df['user']=='Hiệp') & (df['report_month'] >= '2023-11-01') & (df['report_month'] <= '2023-12-31')
df.loc[flt, 'mtd_actual'] = df.loc[flt, 'mtd_Burpee'] + df.loc[flt, 'mtd_non_burpee']

flt = (df['user']=='Hiệp') & (df['report_month'] >= '2023-11-01') & (df['report_month'] <= '2023-12-31') & (df['mtd_non_burpee'] > df['mtd_Burpee']*2)
df.loc[flt, 'mtd_actual'] = df.loc[flt, 'mtd_Burpee']*3


In [26]:

# chinh rule cho Quynh tu thang 6/2023
flt = (df["user"]=="Quỳnh") & (df['report_month']>='2023-06-01') 
df.loc[flt, 'mtd_actual'] = df.loc[flt,'mtd_Burpee'] + df.loc[flt,'mtd_non_burpee']

flt = (df["user"]=="Quỳnh") & (df['report_month']>='2023-06-01') \
    & ((df['mtd_Run']*20+df['mtd_Core']/2+df['mtd_Squat']/3)>(df['mtd_Burpee']+df['mtd_Pushup']/2))
df.loc[flt, 'mtd_actual'] = (df.loc[flt, 'mtd_Burpee'] + df.loc[flt, 'mtd_Pushup']/2)*2.5

# get du lieu tu 1/2023 tro ve truoc

In [76]:
spreadsheet_id = '1Bmm9GGP3QE5k_T8eODN3Hrh8ICzMgAazYkw0gWNRZDs'
sheets = [
        # '1/2023',
        #   '12/2022',
        #   '11/2022',
          '10/2022',
          '9/2022',
          '8/2022',
          '7/2022',
          '6/2022',
          '5/2022',
          ]
gc = pygsheets.authorize(service_file=sf_path)
sh = gc.open_by_key(spreadsheet_id)

for sheet_name in sheets:
    wks = sh.worksheet_by_title(sheet_name)
    df = wks.get_as_df(start='A3', numerize=False, value_render='UNFORMATTED_VALUE', empty_value=None)
    df = df.replace(to_replace='', value=None)
    # get daily
    df_daily = df.iloc[7:,:]
    df_daily = df_daily.melt(id_vars='Ngày')
    df_daily.dropna(subset='value', inplace=True)
    df_daily.rename(columns={'Ngày':'report_date', 'variable':'user', 'value':'Burpee'}, inplace=True)
    df_daily['report_date'] = df_daily['report_date'].astype('str') + '/' + sheet_name
    df_daily['report_date'] = pd.to_datetime(df_daily['report_date'], format='%d/%m/%Y')
    df_daily['Total'] = df_daily['Burpee']
    # get target
    df_target = df.iloc[:7,:]
    df_target

    flt = df_target['Ngày'].isin(['Target', 'Daily'])
    df_target = df_target[flt]
    df_target_clean = df_target.transpose()
    df_target_clean.columns = df_target_clean.iloc[0,:]
    if 'Daily' not in df_target_clean.columns:
        df_target_clean['Daily'] = 0
    df_target_clean = df_target_clean.reset_index()
    df_target_clean =df_target_clean.iloc[1:]
    df_target_clean['report_month'] = '01/' + sheet_name
    df_target_clean['report_month'] = pd.to_datetime(df_target_clean['report_month'], format='%d/%m/%Y')
    df_target_clean.rename(columns={'index':'user'}, inplace=True)
    df_target_clean.reset_index(drop=True,inplace=True)
    df_target_clean
    # Get start of month and number of days in the month
    start_of_month = df_target_clean.loc[0,'report_month']
    num_days = start_of_month.days_in_month


    # Create a date range for each day of the month
    date_range = pd.date_range(start_of_month, periods=num_days, freq='D')
    df_target_clean['report_date'] = [date_range]*len(df_target_clean)
    df_target_clean = df_target_clean.explode('report_date')
    df_target_clean['report_day'] = df_target_clean['report_date'].dt.day
    df_target_clean['no_of_day'] = num_days
    df_target_clean['Daily'] = df_target_clean['Daily'].astype(int)

    df_target_clean['Target'] = df_target_clean['Target'].astype(int)
    df_target_clean['daily_target_norm'] =  df_target_clean['Target']/df_target_clean["no_of_day"]
    df_target_clean['mtd_target_norm'] = df_target_clean['daily_target_norm'] * df_target_clean['report_day']
    dta = df_target_clean.merge(df_daily, how='left', on=['user','report_date'])
    dta['Total'] = dta['Total'].fillna(0)
    dta['Total'] = pd.to_numeric(dta['Total'], errors='coerce')
    dta['mtd_actual'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Total'].cumsum()
    # danh dau ngay dai daily target
    dta['flg_daily']  = 0
    flt = dta['Total'] >= dta['Daily']
    dta.loc[flt, 'flg_daily'] = 1

    # danh dau ngay co tap luyen
    dta['flg_workout'] = 0
    flt = dta['Total'] >0
    dta.loc[flt, 'flg_workout'] = 1


    for col in ['Burpee', 'Core','Pushup', 'Run', 'Squat','Plank','flg_daily', 'flg_workout']:
        try:
            dta[col] = pd.to_numeric(dta[col], errors='coerce')
            dta[col] = dta[col].fillna(0)
            new_col = 'mtd_'+col
            dta[new_col] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])[col].cumsum()
        except Exception as e:
            print(e)
        dta.to_parquet(rf"data/daily/dta_{str(sheet_name).replace('/', '-')}.parquet", index=False )
        print(sheet_name)
        print(dta.groupby(by='report_date')['Total'].sum())

/tmp/ipykernel_3893/3750649981.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dta['Total'] = dta['Total'].fillna(0)


10/2022
report_date
2022-10-01      0.0
2022-10-02      0.0
2022-10-03    260.0
2022-10-04     50.0
2022-10-05    338.0
2022-10-06     50.0
2022-10-07    150.0
2022-10-08    100.0
2022-10-09    117.0
2022-10-10    165.0
2022-10-11      0.0
2022-10-12    130.0
2022-10-13    201.0
2022-10-14    110.0
2022-10-15     70.0
2022-10-16    200.0
2022-10-17    150.0
2022-10-18     60.0
2022-10-19    255.0
2022-10-20    100.0
2022-10-21    110.0
2022-10-22    230.0
2022-10-23    458.0
2022-10-24    171.0
2022-10-25    320.0
2022-10-26    196.0
2022-10-27    317.0
2022-10-28    220.0
2022-10-29     85.0
2022-10-30    320.0
2022-10-31    826.0
Name: Total, dtype: float64
'Core'
10/2022
report_date
2022-10-01      0.0
2022-10-02      0.0
2022-10-03    260.0
2022-10-04     50.0
2022-10-05    338.0
2022-10-06     50.0
2022-10-07    150.0
2022-10-08    100.0
2022-10-09    117.0
2022-10-10    165.0
2022-10-11      0.0
2022-10-12    130.0
2022-10-13    201.0
2022-10-14    110.0
2022-10-15     70.0
2022-

/tmp/ipykernel_3893/3750649981.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dta['Total'] = dta['Total'].fillna(0)


9/2022
report_date
2022-09-01      0.0
2022-09-02      0.0
2022-09-03    260.0
2022-09-04    405.0
2022-09-05    216.0
2022-09-06    355.0
2022-09-07     60.0
2022-09-08    280.0
2022-09-09     75.0
2022-09-10     90.0
2022-09-11    110.0
2022-09-12    260.0
2022-09-13    165.0
2022-09-14    105.0
2022-09-15    185.0
2022-09-16    145.0
2022-09-17    427.0
2022-09-18    490.0
2022-09-19    508.0
2022-09-20    344.0
2022-09-21    300.0
2022-09-22    481.0
2022-09-23    140.0
2022-09-24    222.0
2022-09-25    240.0
2022-09-26    367.0
2022-09-27    525.0
2022-09-28    458.0
2022-09-29    632.0
2022-09-30    741.0
Name: Total, dtype: float64
'Core'
9/2022
report_date
2022-09-01      0.0
2022-09-02      0.0
2022-09-03    260.0
2022-09-04    405.0
2022-09-05    216.0
2022-09-06    355.0
2022-09-07     60.0
2022-09-08    280.0
2022-09-09     75.0
2022-09-10     90.0
2022-09-11    110.0
2022-09-12    260.0
2022-09-13    165.0
2022-09-14    105.0
2022-09-15    185.0
2022-09-16    145.0
2022-09

/tmp/ipykernel_3893/3750649981.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dta['Total'] = dta['Total'].fillna(0)


...

/tmp/ipykernel_3893/3750649981.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dta['Total'] = dta['Total'].fillna(0)


7/2022
report_date
2022-07-01      0.0
2022-07-02      0.0
2022-07-03    445.0
2022-07-04    106.0
2022-07-05    259.0
2022-07-06    143.0
2022-07-07    246.0
2022-07-08    112.0
2022-07-09    178.0
2022-07-10    101.0
2022-07-11    171.0
2022-07-12    221.0
2022-07-13    275.0
2022-07-14    130.0
2022-07-15     60.0
2022-07-16    502.0
2022-07-17    290.0
2022-07-18    172.0
2022-07-19    105.0
2022-07-20    300.0
2022-07-21    300.0
2022-07-22    268.0
2022-07-23    400.0
2022-07-24     81.0
2022-07-25    213.0
2022-07-26    120.0
2022-07-27    212.0
2022-07-28    114.0
2022-07-29    200.0
2022-07-30    440.0
2022-07-31    120.0
Name: Total, dtype: float64
'Core'
7/2022
report_date
2022-07-01      0.0
2022-07-02      0.0
2022-07-03    445.0
2022-07-04    106.0
2022-07-05    259.0
2022-07-06    143.0
2022-07-07    246.0
2022-07-08    112.0
2022-07-09    178.0
2022-07-10    101.0
2022-07-11    171.0
2022-07-12    221.0
2022-07-13    275.0
2022-07-14    130.0
2022-07-15     60.0
2022-07

/tmp/ipykernel_3893/3750649981.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dta['Total'] = dta['Total'].fillna(0)


6/2022
report_date
2022-06-01      0.0
2022-06-02      0.0
2022-06-03      0.0
2022-06-04    270.0
2022-06-05    365.0
2022-06-06    136.0
2022-06-07    300.0
2022-06-08     55.0
2022-06-09    165.0
2022-06-10     75.0
2022-06-11    190.0
2022-06-12    337.0
2022-06-13      0.0
2022-06-14    130.0
2022-06-15    350.0
2022-06-16    195.0
2022-06-17    154.0
2022-06-18    215.0
2022-06-19    277.0
2022-06-20    177.0
2022-06-21    254.0
2022-06-22    327.0
2022-06-23    222.0
2022-06-24    120.0
2022-06-25    395.0
2022-06-26    394.0
2022-06-27    182.0
2022-06-28    172.0
2022-06-29    448.0
2022-06-30    308.0
Name: Total, dtype: float64
'Core'
6/2022
report_date
2022-06-01      0.0
2022-06-02      0.0
2022-06-03      0.0
2022-06-04    270.0
2022-06-05    365.0
2022-06-06    136.0
2022-06-07    300.0
2022-06-08     55.0
2022-06-09    165.0
2022-06-10     75.0
2022-06-11    190.0
2022-06-12    337.0
2022-06-13      0.0
2022-06-14    130.0
2022-06-15    350.0
2022-06-16    195.0
2022-06

/tmp/ipykernel_3893/3750649981.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dta['Total'] = dta['Total'].fillna(0)
